In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
import pandas as pd

from IPython import get_ipython # type: ignore
import os

torch.manual_seed(0)

# get the notebook name
ip = get_ipython()
path = None
if '__vsc_ipynb_file__' in ip.user_ns: # type: ignore
    path = ip.user_ns['__vsc_ipynb_file__'] # type: ignore

os.makedirs('models/', exist_ok=True)
model_file_name = f"models/{os.path.basename(path)[:-6]}.pt" # type: ignore

Data

In [2]:
movie_path = 'dataset/ml-latest-small/movies_with_wikidata_values.csv'
rating_path = 'dataset/ml-latest-small/ratings.csv'

director_df_path = 'dataset/ml-latest-small/directors.csv'
actors_df_path = 'dataset/ml-latest-small/actors.csv'
companies_df_path = 'dataset/ml-latest-small/companies.csv'
countries_df_path = 'dataset/ml-latest-small/countries.csv'

directed_by_path = 'dataset/ml-latest-small/directed_by.csv'
stars_path = 'dataset/ml-latest-small/stars.csv'
produced_by_path = 'dataset/ml-latest-small/produced_by.csv'
produced_in_path = 'dataset/ml-latest-small/produced_in.csv'

movies_df = pd.read_csv(movie_path, index_col='movieId')
ratings_df = pd.read_csv(rating_path)[["userId", "movieId", "rating"]]

directors_df = pd.read_csv(director_df_path, index_col='name')
actors_df = pd.read_csv(actors_df_path, index_col='name')
companies_df = pd.read_csv(companies_df_path, index_col='name')
countries_df = pd.read_csv(countries_df_path, index_col='name')

directed_by_df = pd.read_csv(directed_by_path)[["movieId", "director"]]
stars_df = pd.read_csv(stars_path)[["movieId", "castMember"]]
produced_by_df = pd.read_csv(produced_by_path)[["movieId", "company"]]
produced_in_df = pd.read_csv(produced_in_path)[["movieId", "country"]]

In [3]:
movie_ids_tensor = torch.LongTensor(range(len(ratings_df['movieId'].unique())))
user_ids_tensor = torch.LongTensor(range(len(ratings_df['userId'].unique())))

director_ids_tensor = torch.LongTensor(range(len(directed_by_df['director'].unique())))
star_ids_tensor = torch.LongTensor(range(len(stars_df['castMember'].unique())))
company_ids_tensor = torch.LongTensor(range(len(produced_by_df['company'].unique())))
country_ids_tensor = torch.LongTensor(range(len(produced_in_df['country'].unique())))

genres_to_id_dict = {}
genres_df_list = []
for movie_id, row in movies_df.iterrows():
    for genre in row['genres'].split('|'):
        if genre not in genres_to_id_dict:
            genre_id = len(genres_to_id_dict)
            genres_to_id_dict[genre] = genre_id
        else:
            genre_id = genres_to_id_dict[genre]
        genres_df_list.append((movie_id, genre_id))

genres_df = pd.DataFrame(genres_df_list, columns=['movieId', 'genre'])
genre_ids_tensor = torch.LongTensor(range(len(genres_to_id_dict)))

In [4]:
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedUserId': pd.RangeIndex(len(unique_user_id))
    })

# Create a mapping from the movieId to a unique consecutive value in the range [0, num_movies]:
unique_movie_id = ratings_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': unique_movie_id,
    'mappedMovieId': pd.RangeIndex(len(unique_movie_id))
    })

unique_director_id = directed_by_df['director'].unique()
unique_director_id = pd.DataFrame(data={
    'director': unique_director_id,
    'mappedDirector': pd.RangeIndex(len(unique_director_id))
    })

unique_star_id = stars_df['castMember'].unique()
unique_star_id = pd.DataFrame(data={
    'castMember': unique_star_id,
    'mappedCastMember': pd.RangeIndex(len(unique_star_id))
    })

unique_company_id = produced_by_df['company'].unique()
unique_company_id = pd.DataFrame(data={
    'company': unique_company_id,
    'mappedCompany': pd.RangeIndex(len(unique_company_id))
    })

unique_country_id = produced_in_df['country'].unique()
unique_country_id = pd.DataFrame(data={
    'country': unique_country_id,
    'mappedCountry': pd.RangeIndex(len(unique_country_id))
    })

# Merge the mappings with the original data frame:
ratings_df = ratings_df.merge(unique_user_id, on='userId')
ratings_df = ratings_df.merge(unique_movie_id, on='movieId')

stars_df = stars_df.merge(unique_movie_id, on='movieId').merge(unique_star_id, on='castMember')
directed_by_df = directed_by_df.merge(unique_movie_id, on='movieId').merge(unique_director_id, on='director')
produced_by_df = produced_by_df.merge(unique_movie_id, on='movieId').merge(unique_company_id, on='company')
produced_in_df = produced_in_df.merge(unique_movie_id, on='movieId').merge(unique_country_id, on='country')
genres_df = genres_df.merge(unique_movie_id, on='movieId')

In [5]:
class CustomEmbedding(torch.nn.Embedding):
    def __init__(self, num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None):
        super(CustomEmbedding, self).__init__(num_embeddings, embedding_dim, padding_idx, max_norm, norm_type, scale_grad_by_freq, sparse, _weight)

    def forward(self, input):
        # Create a mask for valid indices
        mask = (input >= 0) & (input < self.num_embeddings)

        # Create a tensor for valid indices, replacing out-of-range indices with 0
        valid_input = torch.where(mask, input, torch.tensor(0, device=input.device))

        # Get the embeddings using the valid input tensor
        embedded = super(CustomEmbedding, self).forward(valid_input)

        # Set embeddings of out-of-range indices to zero
        embedded[~mask] = 0

        return embedded

class GraphEmbedding(torch.nn.Module):
    def __init__(self, user_embedding_dim, movie_embedding_dim, meta_embedding_dim):
        super().__init__()
        self.user_embedding = CustomEmbedding(len(user_ids_tensor), user_embedding_dim)
        self.movie_embedding = CustomEmbedding(len(movie_ids_tensor), movie_embedding_dim)
        self.director_embedding = CustomEmbedding(len(director_ids_tensor), meta_embedding_dim)
        self.star_embedding = CustomEmbedding(len(star_ids_tensor), meta_embedding_dim)
        self.genre_embedding = CustomEmbedding(len(genre_ids_tensor), meta_embedding_dim)
        self.company_embedding = CustomEmbedding(len(company_ids_tensor), meta_embedding_dim)
        self.country_embedding = CustomEmbedding(len(country_ids_tensor), meta_embedding_dim)
        self.user_embedding_dim = user_embedding_dim
        self.movie_embedding_dim = movie_embedding_dim
        self.meta_embedding_dim = meta_embedding_dim

    def forward(self, x_dict):
        x_dict['user'] = self.user_embedding(x_dict['user'])
        x_dict['movie'] = self.movie_embedding(x_dict['movie'])
        x_dict['director'] = self.director_embedding(x_dict['director'])
        x_dict['star'] = self.star_embedding(x_dict['star'])
        x_dict['genre'] = self.genre_embedding(x_dict['genre'])
        x_dict['company'] = self.company_embedding(x_dict['company'])
        x_dict['country'] = self.country_embedding(x_dict['country'])
        return x_dict

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=0.1)
        self.layer_norm = torch.nn.LayerNorm(out_channels)
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)
        self.conv3 = SAGEConv((-1, -1), out_channels)
        self.conv4 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.dropout(x)
        x1 = self.layer_norm(self.conv1(x, edge_index).relu())
        x2 = x1 + self.layer_norm(self.conv2(x1, edge_index).relu())
        x3 = x2 + self.layer_norm(self.conv3(x2, edge_index).relu())
        x4 = x3 + self.layer_norm(self.conv4(x3, edge_index).relu())
        return x4

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = torch.nn.Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels, user_dim, movie_dim, meta_dim, train_data_metadata):
        super().__init__()
        self.graph_embedding = GraphEmbedding(user_dim, movie_dim, meta_dim)
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, train_data_metadata, aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        x_dict = self.graph_embedding(x_dict)
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [6]:
edge_index = torch.stack([
torch.tensor(ratings_df['mappedUserId'].values),
torch.tensor(ratings_df['mappedMovieId'].values)]
, dim=0)

directed_by_edge_index = torch.stack([
torch.tensor(directed_by_df['mappedMovieId'].values),
torch.tensor(directed_by_df['mappedDirector'].values)]
, dim=0)

stars_edge_index = torch.stack([
torch.tensor(stars_df['mappedMovieId'].values),
torch.tensor(stars_df['mappedCastMember'].values)]
, dim=0)

genres_edge_index = torch.stack([
torch.tensor(genres_df['mappedMovieId'].values),
torch.tensor(genres_df['genre'].values)]
, dim=0)

produced_by_edge_index = torch.stack([
torch.tensor(produced_by_df['mappedMovieId'].values),
torch.tensor(produced_by_df['mappedCompany'].values)]
, dim=0)

produced_in_edge_index = torch.stack([
torch.tensor(produced_in_df['mappedMovieId'].values),
torch.tensor(produced_in_df['mappedCountry'].values)]
, dim=0)

assert edge_index.shape == (2, len(ratings_df))

In [7]:
def append_values(tensor, user, values_list):
    if not isinstance(values_list, list):
        values_list = [values_list]
    values_list = [[user, value] for value in values_list]
    new_tensor = tensor.clone()
    for values in values_list:
        if values[0] is not None and values[1] is not None:
            values = torch.tensor(values).view(-1, 1)
            new_tensor = torch.cat((new_tensor, values), dim=1)
    return new_tensor

In [8]:
def append_to_data(movie, user, director=[], star=[], genre=[], company=[], country=[]):
    data = HeteroData()
    # Add the user nodes:
    data['user'].x = torch.LongTensor(range(len(ratings_df['userId'].unique()) + 1)) # [num_users + 1]
    # Add the movie nodes:
    data['movie'].x = torch.LongTensor(range(len(ratings_df['movieId'].unique()) + 1))  # [num_movies + 1]
    # Add the directors nodes:
    data['director'].x = director_ids_tensor # [num_directors]
    # Add the stars nodes:
    data['star'].x = star_ids_tensor # [num_stars]
    # Add the genres nodes:
    data['genre'].x = genre_ids_tensor # [num_genres]
    # Add the company nodes:
    data['company'].x = company_ids_tensor # [num_companies]
    # Add the country nodes:
    data['country'].x = country_ids_tensor # [num_countries]

    # Add the rating edges:
    data['user', 'rates', 'movie'].edge_index = append_values(edge_index, user, movie)  # [2, num_ratings]

    # Add the directed by edges:
    data['movie', 'directed_by', 'director'].edge_index = append_values(directed_by_edge_index, movie, director)
    # Add the stars edges:
    data['movie', 'stars', 'star'].edge_index = append_values(stars_edge_index, movie, star)
    # Add the genre edges:
    data['movie', 'has_genre', 'genre'].edge_index = append_values(genres_edge_index, movie, genre)
    # Add the company edges:
    data['movie', 'produced_by', 'company'].edge_index = append_values(produced_by_edge_index, movie, company)
    # Add the country edges:
    data['movie', 'produced_in', 'country'].edge_index = append_values(produced_in_edge_index, movie, country)

    data = T.ToUndirected()(data)


    del data['movie', 'rev_rates', 'user'].edge_label

    return data

In [9]:
movies_df.head()

,title,genres,year,duration,boxOfficeWorldwide,tommatometerScore
movieId,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,81.0,394436586.0,100%
2,Jumanji (1995),Adventure|Children|Fantasy,1996.0,101.0,262800000.0,52%
3,Grumpier Old Men (1995),Comedy|Romance,1996.0,97.0,NaN,17%
4,Waiting to Exhale (1995),Comedy|Drama|Romance,1996.0,127.0,NaN,60%
5,Father of the Bride Part II (1995),Comedy,1995.0,101.0,NaN,50%


In [10]:
new_movie_idx = len(ratings_df['movieId'].unique())
new_user_idx = len(ratings_df['userId'].unique())
print(new_movie_idx, new_user_idx)

9724 610


In [11]:
non_indexed_movies_df = movies_df.reset_index()
def movie2unique_id(name):
    movieId = non_indexed_movies_df[non_indexed_movies_df['title'] == name].iloc[0]['movieId'].item()
    return int(ratings_df[ratings_df.loc[:, 'movieId'] == movieId].iloc[0]['mappedMovieId'].item())

def name2unique_id(name, meta_df, mapping_df):
    uri = meta_df.loc[name, 'wikidataURI']
    return mapping_df[mapping_df.iloc[:, 1] == uri].iloc[0, 3]

def make_list(value) -> list:
    if value is None:
        return []
    if not isinstance(value, list):
        return [value]
    return value

In [12]:
def demo_prediction(movie=None, user_idx=None, director=None, star=None, genre=None, company=None, country=None, device: str = 'cuda'):
    if movie is None:
        movie_idx = new_movie_idx
    else:
        movie_idx = movie2unique_id(movie)
    if user_idx is None:
        user_idx = new_user_idx
    director = [name2unique_id(d, directors_df, directed_by_df) for d in make_list(director)]
    star = [name2unique_id(s, actors_df, stars_df) for s in make_list(star)]
    genre = [genres_to_id_dict[g] for g in make_list(genre)]
    company = [name2unique_id(c, companies_df, produced_by_df) for c in make_list(company)]
    country = [name2unique_id(c, countries_df, produced_in_df) for c in make_list(country)]

    new_data = append_to_data(movie_idx, user_idx, director=director, star=star, genre=genre, company=company, country=country)
    model = Model(hidden_channels=64, user_dim=512, movie_dim=512, meta_dim=512, train_data_metadata=new_data.metadata()).to(device)
    model.load_state_dict(torch.load(model_file_name))

    with torch.no_grad():
        model.eval()
        new_data = new_data.to(device)
        predictions = model(new_data.x_dict, new_data.edge_index_dict, new_data['user', 'movie'].edge_index)

    return predictions[-1].item()

In [13]:
demo_prediction(
    user_idx = 603,
    director = 'Quentin Tarantino',
    star = ['Leonardo DiCaprio', 'Margot Robbie'],
    genre = ['Action', 'Adventure', 'Western'],
    country = 'United States of America',
    device = 'cpu'
)

3.9246318340301514

In [14]:
demo_prediction(
    user_idx = 603,
    star = ['Tim Robbins', 'Morgan Freeman'],
    genre = ['Drama'],
    director = 'David Fincher',
    device = 'cpu',
)

4.252631664276123

In [15]:
demo_prediction(
    movie = None,
    user_idx = None,
    star = ['Sigourney Weaver', 'Morgan Freeman'],
    director= 'David Fincher',
    device = 'cpu',
    genre = 'Action'
)

3.8620450496673584